In [1]:
import pandas
import datetime
import numpy
from sklearn import linear_model
import scipy
from scipy.optimize import curve_fit
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import ListedColormap
from matplotlib.colors import Normalize
from matplotlib import ticker
from matplotlib import rcParams

In [2]:
# Some parameters we'll use
probCutOff = 0.2
modelVelScale = [ 0., 2000. ]
velsNormArr = numpy.arange(0.,2100.,100.) / max( modelVelScale )
dstVal = -50.

In [3]:
# SAPS Prob Pred Func
def saps_prob_pred_func(MLAT, normMLT, dst):    
    (a_sx, b_sx, a_sy, b_sy, a_xo, b_xo, a_yo, b_yo, a_o, b_o, theta) = \
        ( 3.11, 3.71e-03, 1.72,\
             8.19e-04, 4.59, 6.33e-02, -1.19, 3.21e-02,
             8.93e-01, -1.47e-03, 6.92e-01)
    x = MLAT - 57.5 # NORMALIZE
    y = normMLT
    sigma_x = a_sx + b_sx * dst
    sigma_y = a_sy + b_sy * dst
    xo = a_xo + b_xo * dst
    yo = a_yo + b_yo * dst
    amplitude = a_o + b_o * dst    
    
    a = (numpy.cos(theta)**2)/(2*sigma_x**2) + (numpy.sin(theta)**2)/(2*sigma_y**2)
    b = -(numpy.sin(2*theta))/(4*sigma_x**2) + (numpy.sin(2*theta))/(4*sigma_y**2)
    c = (numpy.sin(theta)**2)/(2*sigma_x**2) + (numpy.cos(theta)**2)/(2*sigma_y**2)
    prob = amplitude*numpy.exp( - (a*((x-xo)**2) + 2*b*(x-xo)*(y-yo) 
                            + c*((y-yo)**2)))
    return prob

In [4]:
# SAPS Vel Azim Pred Func
def saps_velAzim_pred_func(normMLT, dst):

    ( a1mlt_azim, a2mlt_azim,\
               b1mlt_azim, b2mlt_azim ) = ( -1.18e+01, -4.55e-02, 4.24e-01, -6.13e-03 )
    
    # model shape parameters
    # mlt
    a_azim = a1mlt_azim + a2mlt_azim * dst
    b_azim = b1mlt_azim + b2mlt_azim * dst
    # func
    azim = a_azim + b_azim*(normMLT)
    
    return azim

In [5]:
# SAPS Vel Magn Pred Funcs
# We need to get the scale, shape and loc
# parameters and then use them get the pdf 
# and then get the mean velocity from the pdf!!!
def saps_vm_scale_pred_func(mlt,mlat, dst):
    
    ( a_ascmlt, b_ascmlt, a_bscmlt, \
         b_bscmlt, a_cscmlt, b_cscmlt,\
         a_asclat, b_asclat, a_bsclat, b_bsclat ) = ( -1.23e-02, -6.02e-05, -2.9e-04,\
                            -1.79e-05, -1.26e-04, -3.32e-06,\
                            3.47e+01, 2.2e-01, -8.09e-01, -9.76e-04 )
    
    
    a_scale_mlt = a_ascmlt + b_ascmlt * dst
    b_scale_mlt = a_bscmlt + b_bscmlt * dst
    c_scale_mlt = a_cscmlt + b_cscmlt * dst
    # mlat
    a_scale_mlat = a_asclat + b_asclat * dst
    b_scale_mlat = a_bsclat + b_bsclat * dst
    # func
    scale = ( a_scale_mlt + b_scale_mlt*(mlt) + c_scale_mlt*(mlt**2) ) * ( a_scale_mlat + b_scale_mlat*(mlat) )
    
    return scale


def saps_vm_shape_pred_func(mlt,mlat, dst):
    
    ( a_ashmlt, b_ashmlt, a_bshmlt, b_bshmlt,\
               a_ashlat, b_ashlat, a_bshlat, b_bshlat ) = ( -4.72e-02, -3.63e-04, -1.59e-03,\
                                     -2.3e-06, -5.61e+03, 1.19e+02, 8.42e+01, -2.05e+00 ) 
    
    # model shape parameters
    # mlt
    a_shape_mlt = a_ashmlt + b_ashmlt * dst
    b_shape_mlt = a_bshmlt + b_bshmlt * dst
    # mlat
    a_shape_mlat = a_ashlat + b_ashlat * dst
    b_shape_mlat = a_bshlat + b_bshlat * dst
    # func
    shape = ( a_shape_mlt + b_shape_mlt*(mlt) ) * ( a_shape_mlat + b_shape_mlat*(mlat) )
    
    return shape

def saps_vm_loc_pred_func(mlt,mlat, dst):
    
    
    ( a_alcmlt, b_alcmlt, a_blcmlt, b_blcmlt,\
               a_alclat, b_alclat, a_blclat, b_blclat ) = ( -2.81e-03, -2.11e-05, -1.01e-04, -5.51e-06,\
                                        -5.39e-01, 1.50e-03, 7.38e-02, -1.33e-04 )
    
    
    # model loc parameters
    # mlt
    a_loc_mlt = a_alcmlt + b_alcmlt * dst
    b_loc_mlt = a_blcmlt + b_blcmlt * dst
    # malt
    a_loc_mlat = a_alclat + b_alclat * dst
    b_loc_mlat = a_blclat + b_blclat * dst
    # func
    loc = ( a_loc_mlt + b_loc_mlt*(mlt) ) * ( a_loc_mlat*numpy.exp(b_loc_mlat*mlat) )
    return loc

def saps_velMagn_mean_pred_func( scale, shape, loc ):
    
    meanNormVel = scipy.stats.skewnorm.mean( shape, loc=loc, scale=scale )
    meanVel = meanNormVel * max( modelVelScale )
    return meanVel

In [6]:
colList = ["normMLT", "normLAT", "prob_pred",\
           "MLT", "MLAT", "dst_index", "shape_pred",\
           "scale_pred", "loc_pred", "mean_vel_pred", "azim_pred"]
sapsPredDF = pandas.DataFrame(columns=colList)
latArr = []
mltArr = []
normLatArr = []
normMltArr = []
dstValArr = []
dstBinArr = []


for x in numpy.arange(50., 65., 0.5):
    for y in range(-11,5):
        currNormLat =  x - 57.5
        normLatArr.append( currNormLat )
        normMltArr.append( y )
        if y > 0:
            mltArr.append( y )
        else:
            mltArr.append( y + 24 )
        latArr.append( x )
        dstValArr.append(dstVal)
        
sapsPredDF["MLT"] = mltArr
sapsPredDF["MLAT"] = latArr
sapsPredDF["normMLT"] = normMltArr
sapsPredDF["normLAT"] = normLatArr
sapsPredDF["dst_index"] = dstValArr



# Now calculate the prob of occ!!
sapsPredDF["prob_pred"] = saps_prob_pred_func( sapsPredDF["MLAT"],\
                                sapsPredDF["normMLT"], sapsPredDF["dst_index"] )
# Only keep those values whose Dst is gt 0.2
sapsPredDF = sapsPredDF[ sapsPredDF["prob_pred"] > 0.2 ].reset_index(drop=True)

# Now calculate the vel azim
sapsPredDF["azim_pred"] = saps_velAzim_pred_func( sapsPredDF["normMLT"], sapsPredDF["dst_index"] )


# Calculate vel mean magn now!
# We'll calculate scale, loc and shape
# Scale
sapsPredDF["scale_pred"] = saps_vm_scale_pred_func( sapsPredDF["normMLT"],\
                                        sapsPredDF["MLAT"], sapsPredDF["dst_index"] )
# loc
sapsPredDF["shape_pred"] = saps_vm_shape_pred_func( sapsPredDF["normMLT"],\
                                        sapsPredDF["MLAT"], sapsPredDF["dst_index"] )
# Shape
sapsPredDF["loc_pred"] = saps_vm_loc_pred_func( sapsPredDF["normMLT"],\
                                        sapsPredDF["MLAT"], sapsPredDF["dst_index"] )
# Finally get to the velocities
sapsPredDF["mean_vel_pred"] = saps_velMagn_mean_pred_func( sapsPredDF["scale_pred"],\
                                        sapsPredDF["shape_pred"], sapsPredDF["loc_pred"] )
# Round off all values to 2 decimal places!
sapsPredDF["prob_pred"] = sapsPredDF["prob_pred"].round(decimals=2)
sapsPredDF["mean_vel_pred"] = sapsPredDF["mean_vel_pred"].round(decimals=2)
sapsPredDF["azim_pred"] = sapsPredDF["azim_pred"].round(decimals=2)
# sapsPredDF.round
# Consider only those velocities whose magn are greater than 100 m/s
# The shape pred should always be positive
sapsPredDF = sapsPredDF[ sapsPredDF["shape_pred"] > 0.].reset_index(drop=True)
# sapsPredDF = sapsPredDF[ sapsPredDF["mean_vel_pred"] > 100.].reset_index(drop=True)
sapsPredDF.head()

,normMLT,normLAT,prob_pred,MLT,MLAT,dst_index,shape_pred,scale_pred,loc_pred,mean_vel_pred,azim_pred
0,-2,-2.5,0.25,22,55.0,-50.0,33.708150,0.187268,0.107860,514.42,-10.99
1,-1,-2.5,0.28,23,55.0,-50.0,35.613112,0.178484,0.098914,482.53,-10.26
2,0,-2.5,0.25,24,55.0,-50.0,37.518075,0.168251,0.089968,448.33,-9.52
3,-3,-2.0,0.25,21,55.5,-50.0,29.504444,0.198687,0.121599,560.08,-11.72
4,-2,-2.0,0.35,22,55.5,-50.0,31.271715,0.191198,0.112287,529.53,-10.99


In [7]:
# Write the DF to a csv for plotting purposes
outCols = ["MLT", "MLAT", "prob_pred", "dst_index", "mean_vel_pred", "azim_pred"]
outFileName = "../data/full-model-sample-dst" + str(int(dstVal)) + ".txt"
sapsPredDF.to_csv(outFileName, columns = outCols, sep=' ', index=False, header=None)
print "saved to-->", outFileName

saved to--> ../data/full-model-sample-dst-50.txt


In [8]:
print sapsPredDF[ (sapsPredDF["shape_pred"] < 0) ]

Empty DataFrame
Columns: [normMLT, normLAT, prob_pred, MLT, MLAT, dst_index, shape_pred, scale_pred, loc_pred, mean_vel_pred, azim_pred]
Index: []


In [9]:
print sapsPredDF[ (sapsPredDF["normMLT"] == -4) ]

    normMLT  normLAT  prob_pred  MLT  MLAT  dst_index  shape_pred  scale_pred  \
7        -4     -1.5       0.21   20  56.0      -50.0   25.576120    0.208904   
13       -4     -1.0       0.30   20  56.5      -50.0   23.415068    0.213112   
20       -4     -0.5       0.41   20  57.0      -50.0   21.254015    0.217320   
27       -4      0.0       0.53   20  57.5      -50.0   19.092962    0.221527   
34       -4      0.5       0.64   20  58.0      -50.0   16.931910    0.225735   
42       -4      1.0       0.74   20  58.5      -50.0   14.770858    0.229943   
49       -4      1.5       0.81   20  59.0      -50.0   12.609805    0.234150   
56       -4      2.0       0.84   20  59.5      -50.0   10.448753    0.238358   
63       -4      2.5       0.82   20  60.0      -50.0    8.287700    0.242566   
70       -4      3.0       0.77   20  60.5      -50.0    6.126648    0.246774   
76       -4      3.5       0.68   20  61.0      -50.0    3.965595    0.250981   
82       -4      4.0       0